Фрмирвоание вопросов по текстам, которые будем использовать для сравнения моделей

In [38]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_openai import OpenAI
import pandas as pd
from tqdm import tqdm
import yaml
import json
import os

In [39]:
# Параметры скрипта

# DOMAIN = 'movie'
# DOMAIN = 'computer'
DOMAIN = 'nature'

In [40]:
with open('secrets.yaml', 'r') as f:
    secrets = yaml.safe_load(f)

openai_key = secrets['openai_key']

Загружаем данные

In [41]:
with open(os.path.join('artifacts', DOMAIN, 'triples_gt.jsonl'), 'r', encoding='utf-8') as f:
    data_text = f.readlines()

df = pd.DataFrame({'sent': [json.loads(x)['sent'] for x in data_text]})
df.head()

,sent
0,Chapleau River is in the James Bay drainage ba...
1,The mountains classification was won by Nicola...
2,"The wreathed hornbill (Rhyticeros undulatus), ..."
3,The podium placings were completed by another ...
4,The Australian swiftlet (Aerodramus terraeregi...


Создаем промпт для экстракции вопросов из текста

In [42]:
q_extract_prompt = """
Use the text provided to find the one most important question that can be answered using information from the text.
The question must be standalone question and reflect the context.
Return the possible questions as a JSON blob with 'question' key.
Use \" insted " to escape qoutes inside JSON text
Text:
"{text}"
Output:"""

создаем объект PromptTemplate

In [43]:
prompt = PromptTemplate.from_template(q_extract_prompt)

инициализируем модель по умолчанию от OpenAI

In [44]:
model = OpenAI(
    temperature=0,
    openai_api_key = openai_key
    )

In [45]:
model.model_name

'gpt-3.5-turbo-instruct'

Создаем чейн для обращения к модели и json-парсером, чтобы работать с выводом модели как со словарем, а не строкой

In [46]:
chain = prompt | model | JsonOutputParser()

In [47]:
question_list = []
for i_text in tqdm(df['sent']):
    question = chain.invoke({'text' : i_text})
    question_list.append(question['question'])

100%|██████████| 157/157 [01:36<00:00,  1.63it/s]


In [48]:
question_list

['What is the main tributary of Kapuskasing Lake?',
 'Who won the mountains classification in the Tour de France?',
 'What is the distribution of the wreathed hornbill species?',
 'Who finished third on Willunga Hill and took four bonus seconds in the 2018 Tour Down Under?',
 'What is the scientific name of the Australian swiftlet?',
 'What is the family of Nesoluma plants?',
 "Who won the Giro Rosa race for the second time in three years and took the overall lead of the UCI Women's World Tour standings?",
 "What is the name of the river that flows through Tai'an, Feicheng, and empties into Dongping Lake?",
 'Where is the feature located in relation to Istind Peak and Ahlmann Ridge in Queen Maud Land?',
 'What is the genus of the only living member of the Long-tailed duck?',
 'Where is Mount Sannosawa located?',
 'What is the location of Ove Peak?',
 'What towns does Pannerdens Kanaal flow past and where does it continue to the sea?',
 'What is the relationship between Fan Frynych and 

In [49]:
df['question'] = question_list
df.to_csv(os.path.join('artifacts', DOMAIN, 'eval_dataset.csv'), index=False)